In [1]:
import torch
from torch.nn import Conv2d, Linear
from torchvision.models import resnet50
import os,sys
parent_dir = os.path.abspath('../')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
resnet = resnet50()

In [ ]:
resnet.conv1 = Conv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet.fc = Linear(in_features=2048, out_features=1024, bias=True)
print(resnet)

In [ ]:
from util.frame_diff import Sandevistan
from util.dark_enhance import DarkEnhance
from torchvision.io import read_video
frames, _, info  = read_video('./Run_1_1.mp4')
print('frames',frames.shape)

In [ ]:
enhancer = DarkEnhance()
#frame_diff = Sandevistan(n_frames=10,thres=1.0)
frames = enhancer(frames).transpose(3,1).unsqueeze(0)
#out = frame_diff(frames.clone())
#print(out.shape)

In [2]:
#from model.network import SandevistanCLIP, build_model
state_dict_new = torch.load('/home/neoncloud/low_light/checkpoints/2022_11_05_04_41_13/model_best.pt')
state_dict_old = torch.load('/home/neoncloud/low_light/test/model_best_modified.pt')
#model_state_dict = state_dict['model_state_dict']
#fusion_state_dict = state_dict['fusion_model_state_dict']
#clip = build_model(state_dict,pretrain=True)
#print(state_dict_old['model_state_dict'].keys())

In [3]:
for k,v in state_dict_new['model_state_dict'].items():
    if 'motion' in k:
        state_dict_old['model_state_dict'][k] = v

In [4]:
torch.save(state_dict_old,'model_best_motion_finetune.pt')

In [ ]:
new_model_state_dict = {}
for k,v in state_dict_new['model_state_dict'].items():
    if 'motion' in k:
        continue
    new_model_state_dict[k] = v
state_dict_new['model_state_dict'] = new_model_state_dict
print(state_dict_new['model_state_dict'].keys())

In [ ]:
print(state_dict_new['model_state_dict'].keys())

In [ ]:
for ko,kn in zip(state_dict_old['model_state_dict'].keys(), state_dict_new['model_state_dict'].keys()):
    if 'fusion' in ko:
        state_dict_new['model_state_dict'][ko] = state_dict_old['model_state_dict'][ko]

In [ ]:
torch.save(state_dict_new,'model_best_modified.pt')

In [ ]:
import yaml
from dotmap import DotMap
with open('/home/neoncloud/low_light/config/low_light_train.yaml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
cfg = DotMap(cfg)

In [ ]:
from model.network import build_model
model = build_model(
            state_dict=model_state_dict,
            pretrain=True,
            motion_layers=cfg.network.motion.num_layers,
            motion_layers_init=cfg.network.motion.init,
            train_visual=cfg.visual.train,
            T=cfg.data.seg_length,
            thres=cfg.network.motion.thres,
            alpha=cfg.network.other.alpha,
            fusion_type=cfg.network.fusion.type
        )

In [ ]:
print(state_dict.keys())

In [ ]:
print(state_dict['fusion_model_state_dict'].keys())

In [ ]:
new_fusion_state_dict={}
for k,v in fusion_state_dict.items():
    k = k.strip('module.')
    if 'transformer' in k:
        k = k.replace('transformer','fusion')
    if 'frame_position_embeddings' in k:
        pass
    new_fusion_state_dict[k] = v

In [ ]:
print(new_fusion_state_dict.keys())

In [ ]:
model.fusion.load_state_dict(new_fusion_state_dict,False)

In [ ]:
torch.save({'model_state_dict': model.state_dict()},'/mnt/e/vit-b-16-8f_modified3.pt')

In [ ]:
motion = {}
for k,v in state_dict.items():
    if 'visual.' in k:
        if k == 'visual.conv1.weight':
            continue
        k_ = k.replace('visual.','motion.')
        motion[k_] = v
state_dict.update(motion)

In [ ]:
print(state_dict.keys())

In [ ]:
from clip.clip import build_model
clip = build_model(model_state_dict).cuda()

In [ ]:
from torch.nn.functional import interpolate
print(frames[0,40,...].shape)
frames = interpolate(frames[0,40,None,None,...],size=(3,224,224)).squeeze(0)
print(frames.shape)
import matplotlib.pyplot as plt
plt.imshow(frames.squeeze(0).transpose(0,2))

In [ ]:
image_token = clip.encode_image(frames.cuda())

In [ ]:
text_list = ['Drink','Jump','Pick','Pour','Push','Run','Sit','Stand','Turn','Walk']
from model.text_prompt import text_prompt
num_text_aug, text_tokenized = text_prompt(text_list)
print(text_tokenized.shape)

In [ ]:
text_token = clip.encode_text(text_tokenized.reshape(-1,77).cuda()).reshape(16,10,-1)
print(text_token.shape)

In [ ]:
image_features = image_token / image_token.norm(dim=1, keepdim=True)
text_features = text_token / text_token.norm(dim=1, keepdim=True)

# cosine similarity as logits
logits_per_text = 100 *  text_features @ image_features.t()

In [ ]:
logits_per_text = logits_per_text.mean(0)
print(logits_per_text)

In [ ]:
logits_per_text = logits_per_text.softmax(0)

In [ ]:
_state_dict = clip.state_dict()

In [ ]:
clip.load_state_dict(new_fusion_state_dict, strict=False)

In [ ]:
torch.save({'model_state_dict': clip.state_dict(), 'optimizer_state_dict':optim_dict},'/mnt/e/vit-b-16-8f_modified2.pt')

In [ ]:
a = torch.rand(6,8,3,224,224).half().cuda()
b = torch.randint(10,(10,77)).cuda()
out = clip.encode_image(a)

In [ ]:
print(out.shape)

In [ ]:
out[0].sum().backward()

In [ ]:
a = torch.zeros(12,12)
b = torch.tensor([[7, 1, 9, 0, 0, 0, 3, 0, 9, 7, 2, 7]],dtype=torch.long)
u,b = torch.unique(b,return_inverse=True,sorted=False)
#u,b = torch.unique(b,return_inverse=True,sorted=False)
print(b)
c = torch.ones(1,12)
a = a.scatter(0,index=b,src=c)
print(a)

In [ ]:
(0.95+0.05*torch.randn(b.shape[1]))*(torch.eq(b,b.T).to(torch.float16))

In [ ]:
# tensor([7, 1, 9, 0, 0, 0, 3, 0, 9, 7, 2, 7])
# tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
#         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
#         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
#         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
#         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.]], device='cuda:0',
#        dtype=torch.float16)

In [ ]:
a = torch.rand(6,8,3,224,224).half().cuda()
out = clip.encode_image(a)
print(out.shape)

In [ ]:
b = torch.randint(10,(160,77)).cuda()
out = clip.encode_text(b)
print(out.dtype)

In [ ]:
print(sys.getsizeof(out.storage()))

In [ ]:
a = torch.tensor([[0,6,3,2,5],[7,4,5,3,0],[0,1,2,4,5]])
print(a.shape)
b = torch.tensor([0,8,2]).unsqueeze(-1)
print(b.shape)
torch.any(torch.eq(a,b),dim=-1)

In [ ]:
a = torch.tensor([0,1,2])
b = torch.tensor([0,8,2])
torch.eq(a,b)